In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('../src')
from functions import *

df_clients_profile = pd.read_csv('../Data/raw/df_final_demo.txt')
df_web_data_1 = pd.read_csv('../Data/raw/df_final_web_data_pt_1.txt')
df_web_data_2 = pd.read_csv('../Data/raw/df_final_web_data_pt_2.txt')
df_experiment_clients = pd.read_csv('../Data/raw/df_final_experiment_clients.txt')
pd.set_option('display.max_columns', None)
df_web_data = pd.concat([df_web_data_1, df_web_data_2], ignore_index= True)

In [ ]:
#lenght of the dataframes
lenght_clients_profile = print(df_clients_profile.shape)
lenght_experiment_clients = print(df_experiment_clients.shape)
lengt_web_data = print(df_web_data.shape)

In [ ]:
#clean df
df_clients_profile = print_clean_data(df_clients_profile)

In [ ]:
#clean df
df_experiment_clients = print_clean_data(df_experiment_clients)

In [ ]:
#clean df
df_web_data = print_clean_data(df_web_data)

In [ ]:
df_null_experiment_clients = df_experiment_clients.isna()
df_null_experiment_clients

In [ ]:
df_experiment_clients

In [ ]:
#setting aside the clients that wont be part if the experiment
null_df_experiment_clients = df_experiment_clients[df_experiment_clients['Variation'].isnull()]
null_df_experiment_clients

In [ ]:
#id of clients that are part of the test
df_experiment_clients_Test = df_experiment_clients[df_experiment_clients['Variation'] == 'Test']
df_experiment_clients_Test.head(10)

In [ ]:
df_test = pd.merge(df_experiment_clients_Test, df_clients_profile, on='client_id', how='inner')
df_test = pd.merge(df_web_data, df_test, on='client_id', how='inner')
#change the data type to datetime
df_test['date_time'] = pd.to_datetime(df_test['date_time']) 

df_test_latest_confirm = df_test[df_test['process_step'] == 'confirm'].sort_values('date_time').drop_duplicates('visit_id', keep='last')

# Combine with other process steps
df_test = pd.concat([df_test_latest_confirm, df_test[df_test['process_step'] != 'confirm']])

# Sort the final DataFrame by the original index and date_time
df_test = df_test.sort_index().sort_values('date_time').reset_index(drop=True)
df_test.sample(5)

In [105]:
df_experiment_clients_Control = df_experiment_clients[df_experiment_clients['Variation'] == 'Control']
df_experiment_clients_Control.head(10)

,client_id,Variation
2,4033851,Control
4,9294070,Control
7,6651403,Control
9,2105948,Control
12,9814849,Control
14,7286739,Control
15,4446011,Control
20,4229407,Control
24,2842481,Control
27,1218661,Control


In [106]:
df_control = pd.merge(df_experiment_clients_Control, df_clients_profile, on='client_id', how='inner')
df_control = pd.merge(df_web_data, df_control, on='client_id', how='inner')
#change the data type to datetime
df_control['date_time'] = pd.to_datetime(df_control['date_time'])
df_latest_confirm = df_control[df_control['process_step'] == 'confirm'].sort_values('date_time').drop_duplicates('visit_id', keep='last')

# Combine with other process steps
df_control = pd.concat([df_latest_confirm, df_control[df_control['process_step'] != 'confirm']])

# Sort the final DataFrame by the original index and date_time
df_control = df_control.sort_index().sort_values('date_time').reset_index(drop=True)

In [108]:
Completion_rate_test = df_test[df_test['process_step'] == 'confirm']['visit_id'].nunique() / df_test['visit_id'].nunique() * 100

In [109]:
Completion_rate_test

58.51734166307626

In [110]:
Completion_rate_control = df_control[df_control['process_step'] == 'confirm']['visit_id'].nunique() / df_test['visit_id'].nunique() * 100
Completion_rate_control

43.20874623007324

In [89]:
def calculate_time_spent(group):
    group = group.sort_values('date_time')
    group['time_spent'] = group['date_time'].diff().fillna(pd.Timedelta(seconds=0))
    time_spent = group.groupby('process_step')['time_spent'].mean().dt.total_seconds()
    return time_spent

time_spent_df = df_test.groupby('visit_id').apply(calculate_time_spent).reset_index()
average_time_spent = time_spent_df.groupby('process_step')['time_spent'].mean()

C:\Users\reetu\AppData\Local\Temp\ipykernel_7760\1827866774.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  time_spent_df = df_test.groupby('visit_id').apply(calculate_time_spent).reset_index()


In [90]:
average_time_spent

process_step
confirm    121.176844
start       34.459363
step_1      33.235002
step_2      41.502085
step_3      92.496816
Name: time_spent, dtype: float64

In [96]:
# Assuming 'previous_step' column indicates the previous step a user was at
df_test['backward_move'] = df_test.groupby('visit_id')['process_step'].shift(1) > df_test['process_step']
error_rate_test = (df_test['backward_move'].sum() / df_test['visit_id'].nunique() * 100).round(2)
error_rate_test

np.float64(92.51)

In [97]:
df_control['backward_move'] = df_control.groupby('visit_id')['process_step'].shift(1) > df_control['process_step']
error_rate_control = (df_control['backward_move'].sum() / df_control['visit_id'].nunique() * 100).round(2)
error_rate_control

np.float64(75.3)

In [101]:
# Number of users reaching each step
users_reaching_step = df_test.groupby('process_step')['visit_id'].nunique()

# Number of users abandoning each step
drop_offs = users_reaching_step - users_reaching_step.shift(-1, fill_value=users_reaching_step[0])

drop_off_rate_test = (drop_offs / users_reaching_step) * 100
drop_off_rate_test

C:\Users\reetu\AppData\Local\Temp\ipykernel_7760\3031222085.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  drop_offs = users_reaching_step - users_reaching_step.shift(-1, fill_value=users_reaching_step[0])


process_step
confirm   -52.579265
start      14.693730
step_1     13.371045
step_2      9.455985
step_3      2.050843
Name: visit_id, dtype: float64

In [102]:
# Number of users reaching each step
users_reaching_step = df_control.groupby('process_step')['visit_id'].nunique()

# Number of users abandoning each step
drop_offs = users_reaching_step - users_reaching_step.shift(-1, fill_value=users_reaching_step[0])

drop_off_rate_control = (drop_offs / users_reaching_step) * 100
drop_off_rate_control

C:\Users\reetu\AppData\Local\Temp\ipykernel_7760\3935763715.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  drop_offs = users_reaching_step - users_reaching_step.shift(-1, fill_value=users_reaching_step[0])


process_step
confirm   -92.633678
start      23.817535
step_1     14.481060
step_2      9.127024
step_3     12.316940
Name: visit_id, dtype: float64

In [103]:
# Calculate the average path length for users who completed the process
path_lengths = df_test.groupby('visit_id')['process_step'].count()
average_path_length_test = path_lengths.mean()
average_path_length_test

np.float64(4.681602757432142)

In [104]:
path_lengths = df_control.groupby('visit_id')['process_step'].count()
average_path_length_control = path_lengths.mean()
average_path_length_control

np.float64(4.4115381030786915)

In [ ]:
# Ensure 'start_times' has unique indices
start_times = df_test[df_test['process_step'] == 'start'].groupby('visit_id')['date_time'].min()

# Map start times to the DataFrame
df_test = df_test.sort_values(by=['visit_id', 'date_time'])
df_test['start_time'] = df_test['visit_id'].map(start_times)

# Calculate time spent in minutes
df_test['time_spent_seconds'] = (df_test['date_time'] - df_test['start_time']).dt.total_seconds() 

# Drop 'start_time' column
df_test= df_test.drop(columns=['start_time'])

# Create a new DataFrame with the final step
df_test_last_step = df_test.sort_values(by=['visit_id', 'date_time']).drop_duplicates(subset=['visit_id'], keep='last')

df_test_last_step[['client_id', 'visit_id', 'process_step', 'date_time', 'time_spent_seconds']].head(10)

In [ ]:
step_counts = df_test_last_step['process_step'].value_counts()
# Plot the counts of each step
plt.figure(figsize=(10, 6))
step_counts.plot(kind='bar', color='skyblue')
plt.title('Count of Each Process Step')
plt.xlabel('Process Step')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [65]:
# Ensure 'start_times' has unique indices
start_times = df_control[df_control['process_step'] == 'start'].groupby('visit_id')['date_time'].min()


# Map start times to the DataFrame
df_control = df_control.sort_values(by=['visit_id', 'date_time'])
df_control['start_time'] = df_control['visit_id'].map(start_times)

# Calculate time spent in minutes
df_control['time_spent_seconds'] = (df_control['date_time'] - df_control['start_time']).dt.total_seconds() 

# Drop 'start_time' column
df_control = df_control.drop(columns=['start_time'])

# Create a new DataFrame with the final step
df_control_last_step= df_control.sort_values(by=['visit_id', 'date_time']).drop_duplicates(subset=['visit_id'], keep='last')

df_control_last_step[['visit_id', 'process_step', 'date_time', 'time_spent_seconds']].head(10)

,visit_id,process_step,date_time,time_spent_seconds
98990,100030127_47967100085_936361,start,2017-03-22 11:07:49,0.0
11353,100037962_47432393712_705583,start,2017-04-14 16:44:03,132.0
644,100057941_88477660212_944512,step_3,2017-04-09 11:34:24,254.0
100305,10006594_66157970412_679648,step_3,2017-04-13 11:56:12,354.0
87269,100071743_53464757454_616703,start,2017-04-07 10:50:20,0.0
138346,10007589_47780784567_391490,confirm,2017-05-18 08:03:33,721.0
105311,100096068_8301717872_987164,step_1,2017-05-22 18:15:14,13.0
50370,100105975_18174738494_196339,start,2017-04-10 06:30:30,13.0
24790,100195539_90558871716_822894,step_1,2017-03-30 10:46:18,363.0
123870,10024125_38177552152_999695,step_1,2017-05-01 09:03:51,21.0


In [ ]:
step_counts = df_control_last_step['process_step'].value_counts()
# Plot the counts of each step
plt.figure(figsize=(10, 6))
step_counts.plot(kind='bar', color='skyblue')
plt.title('Count of Each Process Step')
plt.xlabel('Process Step')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:

control_counts = df_control_last_step['process_step'].value_counts()
test_counts = df_test_last_step['process_step'].value_counts()

# Combine the counts into a single DataFrame
combined_counts = pd.DataFrame({
    'Control': control_counts,
    'Test': test_counts
}).fillna(0)

# Define the desired order of process steps
desired_order = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

# Reindex the DataFrame to reflect the desired order
combined_counts = combined_counts.reindex(desired_order)

# Plot the combined counts
plt.figure(figsize=(12, 8))
combined_counts.plot(kind='bar', color=['skyblue', 'lightgreen'])
plt.title('Count of Each Process Step (Control vs Test)')
plt.xlabel('Process Step')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(title='Dataset')
plt.tight_layout()
plt.show()

# Day 1 & 2 (Week 5)

Answer the following questions about demographics:

Who are the primary clients using this online process?
Are the primary clients younger or older, new or long-standing?


In [ ]:
result = df_test[df_test['visit_id'] == '100217156_67053490690_383412']
result

In [ ]:
# Displaying the data types of each column in the dataframe
df_test.dtypes

In [ ]:
# Displaying the data types of each column in the dataframe
df_control.dtypes

In [ ]:
# Retrieving the unique data types present in the df_test
list(set(df_test.dtypes.tolist()))

In [ ]:
# Retrieving the unique data types present in the df_control
list(set(df_control.dtypes.tolist()))

In [ ]:
# Extracting column names with numerical data types from the dataframe
df_test.select_dtypes("number").columns

In [ ]:
# Counting and sorting the unique values for each numerical column in descending order
df_test.select_dtypes("number").nunique().sort_values(ascending=False)

In [ ]:
# Extracting column names with numerical data types from the dataframe
df_control.select_dtypes("number").columns

In [ ]:
# Counting and sorting the unique values for each numerical column in descending order
df_control.select_dtypes("number").nunique().sort_values(ascending=False)